## Brain observatory
This notebook documents some classes and functions in the [Allen Software Development Kit](http://alleninstitute.github.io/AllenSDK/) (SDK) that help manipulate files and data structures in the Allen Brain Observatory. The main entry point in the BrainObservatoryCache class. This class is responsible for downloading any requested data or metadata on request and storing it in well known locations. Detailed information about the experimental parameters is available in [technical white papers](http://help.brain-map.org/display/observatory/Documentation).

### Experiment containers
The experiment container describes a set of experiments performed with the same targeted area, imaging depth, and Cre line. The BrainObservatoryCache has a number of functions for figuring out what experiment containers are available at the moment.

In [ ]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import pprint

# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

# Download a list of all targeted areas
targeted_structures = boc.get_all_targeted_structures()
print("all targeted structures: " + str(targeted_structures))

In [ ]:
# Download experiment containers for VISp experiments
visp_ecs = boc.get_experiment_containers(targeted_structures=['VISp'])
print("all VISp experiment containers: %d" % len(visp_ecs))

In [ ]:
# Download a list of all imaging depths
depths = boc.get_all_imaging_depths()
print("all imaging depths: " + str(depths))

In [ ]:
# Download a list of all stimuli
stims = boc.get_all_stimuli()
print("all stimuli:\n")
pprint.pprint(stims)

In [ ]:
# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print("all cre lines: " + str(cre_lines))

In [ ]:
# Download experiment containers for Cux2 experiments
cux2_ecs = boc.get_experiment_containers(cre_lines=['Cux2-CreERT2'])
print("Cux2 experiments: %d\n" % len(cux2_ecs))

print("Example experiment container record:")
pprint.pprint(cux2_ecs[0])

### Download experiments for a container
An experiment container is a group of experiments. Each experiment has a different stimulus protocol. For example, one experiment protocol contains the static gratings stimulus and another has the natural scenes stimulus. The BrainObservatoryCache helps you find out which experiment associated with a container has the stimuli you are interested in. First, let's see what experiments are available for a single container.

In [ ]:
# Find all of the experiments for an experiment container
cux2_ec_id = cux2_ecs[0]['id']
exps = boc.get_ophys_experiments(experiment_container_ids=[cux2_ec_id])
print("Experiments for experiment_container_id %d: %d\n" % (cux2_ec_id, len(exps)))
pprint.pprint(exps)

The session_type field indicates which experimental protocol was used. If you just want to find which experiment contains the static gratings stimulus, you can do the following:

In [ ]:
import allensdk.brain_observatory.stimulus_info as stim_info

# Find the experiment with the static static gratings stimulus
cux2_ec_id = cux2_ecs[0]['id']
exp = boc.get_ophys_experiments(experiment_container_ids=[cux2_ec_id], 
                                stimuli=[stim_info.STATIC_GRATINGS])[0]
print("Experiment with static gratings:")
pprint.pprint(exp)

Now we can download the NWB file for this experiment. The file will be stored in the folder boc/ophys_experiment_data/. The file format is [Neurodata without Borders](http://www.nwb.org/) (.nwb), which is essentially an HDF5 file that can be conveniently viewed using appropriate viewers, such as [HDFView](https://www.hdfgroup.org/products/java/hdfview/). Note that the download can take some time.

In [ ]:
exp = boc.get_ophys_experiment_data(exp['id'])

# print out the metadata available in the NWB file
pprint.pprint(exp.get_metadata())

We can check which files have been downloaded using standard Linux shell commands. These can be used in a notebook cell by prefacing %%bash to the cell:

In [ ]:
%%bash
ls -lsah boc/ophys_experiment_data/

### Extract traces from experiment

In [ ]:
# several different traces are available (raw, neuropil corrected, Dff)
time, raw_traces = exp.get_fluorescence_traces()
time, corrected_traces = exp.get_corrected_fluorescence_traces()
time, dff_traces = exp.get_dff_traces()

### Plot traces

In [ ]:
# we use the Bokeh library because of the available interactive tools (zoom, data cursor etc.)
from bokeh.plotting import Figure, show
from bokeh.models import Range1d, CrosshairTool, HoverTool
from bokeh.io import output_notebook

import numpy as np

In [ ]:
output_notebook()

In [ ]:
def getHover():
    """Define and return hover tool for a plot"""
    # Define hover tool
    hover = HoverTool()
    hover.tooltips = [
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
        ("fill color", "$color[hex, swatch]:fill_color"),
    ]
    return hover

In [ ]:
def plotTimeseries(p, t, y):
    """
    Plot a timeseries in Figure p using the Bokeh library
    
    Input arguments:
    p ... Bokeh figure
    t ... 1d time axis vector (numpy array)
    y ... 1d data vector (numpy array)
    """
    
    p.add_tools(CrosshairTool(), getHover())
    p.line(t, y, line_width=2, color='blue')
    left, right = np.min(t), np.max(t)
    p.set(x_range=Range1d(left, right))
    show(p)
    
    return p

In [ ]:
# plot raw fluorescence trace
p = Figure(plot_width=800, plot_height=300, title="Raw Fluorescence Trace")
p = plotTimeseries(p, time, raw_traces[0])

# plot neuropil-corrected fluorescence trace
p = Figure(plot_width=800, plot_height=300, title="Neuropil-corrected Fluorescence Trace")
p = plotTimeseries(p, time, corrected_traces[0])

# plot dF/F trace
p = Figure(plot_width=800, plot_height=300, title="dF/F Trace")
p = plotTimeseries(p, time, dff_traces[0])

### Running speed
The animal's running speed during the course of the experiment is also stored in the NWB file.

In [ ]:
dxcm, dxtime = exp.get_running_speed()
p = Figure(plot_width=800, plot_height=300, title="Running speed")
p = plotTimeseries(p, dxtime, dxcm)

### ROI Masks
If you want to take a look at the cell visually, you can open the NWB file and extract a pixel mask. You can also pull out the maximum intensity projection of the movie for context.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# get the specimen IDs for a few cells
cids = exp.get_cell_specimen_ids()[:15:5]

# get masks for specific cells
roi_mask_list = exp.get_roi_mask(cell_specimen_ids=cids)

# plot each mask
f, axes = plt.subplots(1, len(cids)+2, figsize=(15, 3))
for ax, roi_mask, cid in zip(axes[:-2], roi_mask_list, cids):
    ax.imshow(roi_mask.get_mask_plane(), cmap='gray')
    ax.set_title('cell %d' % cid)

# make a mask of all ROIs in the experiment    
all_roi_masks = exp.get_roi_mask()
sum_mask = np.zeros(exp.MOVIE_FOV_PX)
for roi_mask in all_roi_masks:
    sum_mask[roi_mask.get_mask_plane()>0] = 1

axes[-2].imshow(sum_mask, cmap='gray')
axes[-2].set_title('all ROIs')

# show the movie max projection
max_projection = exp.get_max_projection()
axes[-1].imshow(max_projection, cmap='gray')
axes[-1].set_title('max projection')

plt.show()